In [1]:
#Limpiza nueva

In [2]:
!pip install --upgrade spacy
!python -m spacy download es_core_news_sm
import spacy

# Cargar el modelo de idioma español de SpaCy
nlp = spacy.load('es_core_news_sm')

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

# Inicializar el tokenizador
tokenizer = ToktokTokenizer()

# Descargar la lista de stopwords en español de NLTK
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('spanish')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 22.6 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


[nltk_data] Downloading package stopwords to /Users/sil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv("../data/capi_chat.csv")
df

,Fecha,Hora,Persona,Mensaje
0,"30/11/23,",19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,"30/11/23,",19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,"30/11/23,",19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,"30/11/23,",19:12:56,Alana Ironhack,Hola
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,"17/2/24,",21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,"17/2/24,",21:11:04,Alana Ironhack,Jajaj
1771,"17/2/24,",21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,"17/2/24,",21:18:17,Sil🐨,JAJAJAJAJA


# Eliminamos acentos

In [4]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

remove_accented_chars('Sómě Áccěntěd těxt')

'Some Accented text'

# cambiamos abreviaturas

In [5]:
# abreviaturas

CONTRACCIONES_ESP = {
"xq": "porque",
"Holiiii": "Hola",
"sticker omitido": "imagen",
"imagen omitida": "imagen",
"q": "que",
"X": "por",
"x": "por",
"esq": "es que",
"GIF omitido": "GIF",
"Valee": "vale",
"perfe": "perfecto",
"Siiii": "Si",
"siii": "Si",
"Ayyy":"Ay",
"proque": "porque",
"nose": "no se",
"tmbn": "tambien",
"lab": "laboratorio",
"labs": "labboratotios",
"lst": "lista",
"reu": "reunion",
"michi": "gato",
"jaajaaj": "jajajajaja",
"xd": "risa",
"capiiis": "capi",
"smpr": "siempre",
"Noup": "No",
"okii": "vale",
"year": "año",
"string": "letras",
"colorinchis": "colores",
"pull request": "subir",
"LOL": "risa a carcajadas",
"Wsp": "WhatsApp",
"Wasap": "WhatsApp",
"wa": "WhatsApp",
"Guasap": "WhatsApp",
"Pq": "porque",
"Xq": "porque",
"q": "que",
"Dnd": "donde",
"K": "que",
"T": "te",
"+/-": "mas o menos",
"Nd": "nada",
"Vdd": "verdad",
"Ntp": "no te preocupes",
"Tqm": "Te quiero mucho",
"Tq": "Te quiero",
"Tmb": "tambien",
"Dps": "despues",
"Xfa": "por favor",
"Dd": "donde",
"Oki": "vale",
"Ok": "vale",
"Tkt": "tranquilo",
"Tqg": "te quiero un monton",
"Bss": "besos",
"Tmbn": "tambien",
"Fvr": "favor",
"Msj": "mensaje",
"LMAO": "risa",
"BRB": "Vuelvo enseguida",
"IDK": "no se",
"idk": "no se",
"ASAP": "tan pronto como sea posible",
"OMG": "Oh Dios mio",
"BFF": "Mejores amigos para siempre",
"Okey": "vale"
    }

In [6]:
def corregir_abreviaturas(texto, mapeo_abreviaturas=CONTRACCIONES_ESP):
    texto_corregido = []
    for palabra in texto.replace(".","").split():
        palabra_corregida = mapeo_abreviaturas.get(palabra.lower(), palabra)
        if palabra_corregida is not None:
            texto_corregido.append(palabra_corregida)
        else:
            texto_corregido.append(palabra)
    return ' '.join(texto_corregido)
# Llamar a la función corregir_abreviaturas con el diccionario de abreviaturas
a = corregir_abreviaturas( 'Q tal. okey xq.')
print(a)

que tal okey porque


# caracteres especiales

In [7]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("¡Bueno, esto fue divertido! ¿Qué opinas? 123#@!", 
                          remove_digits=True)

'Bueno esto fue divertido Qu opinas '

# Text lemmatization

In [8]:
# Importar NLTK
import nltk
from nltk.stem import SnowballStemmer

# Crear una instancia de SnowballStemmer para español
stemmer_es = SnowballStemmer('spanish')

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text('Mi sistema sigue fallando. El suyo falló ayer, el nuestro falla diariamente.')

'mi sistema seguir fallar . el suyo fallar ayer , el nuestro falla diariamente .'

# Text stemming

In [9]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

simple_stemmer('Mi sistema sigue fallando. El suyo falló ayer, el nuestro falla diariamente.')

'mi sistema sigu fallando. el suyo falló ayer, el nuestro falla diariamente.'

# Remove stopwords

In [10]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("De, y, si son palabras vacías, la computadora no lo es")

', , si palabras vacías , computadora'

# Building a text normalizer

In [11]:
def normalize_corpus(corpus, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = corregir_abreviaturas(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus


# Pre-process and normalize news articles

In [12]:
df["Mensaje"]

0       ‎Los mensajes y las llamadas están cifrados de...
1       ‎Alana Ironhack creó el grupo “Capibaras Ironh...
2                              ‎Alana Ironhack te añadió.
3                                                    Hola
4       Les paso las fotos de ayer prepárense xq es un...
                              ...                        
1769    No y que también pensaba que esos días que íba...
1770                                                Jajaj
1771                             Disfrutar de tu libertad
1772                                           JAJAJAJAJA
1773                                                  Así
Name: Mensaje, Length: 1774, dtype: object

In [13]:
df['clean_text'] = normalize_corpus(df['Mensaje'])
#norm_corpus = list(df['Mensaje'])
df.iloc[1][['Mensaje', 'clean_text']].to_dict()
#news_df.iloc[1]['clean_text'].to_dict()

{'Mensaje': '\u200eAlana Ironhack creó el grupo “Capibaras Ironhack”.',
 'clean_text': 'aln ironhack creer grupo capibara ironhack'}

# exportar datos limpios

In [14]:
# Exportamos los datos limpios en un archivo csv
df.to_csv('../data/data_capi_chat_clear.csv', header=True, index=False)

# analisis con TextBlob

In [15]:
from textblob import TextBlob

In [16]:
mensajes = df['clean_text'].tolist()

# Lista para almacenar los sentimientos de cada mensaje
sentimientos = []

# Iterar sobre cada mensaje y calcular el sentimiento
for texto in mensajes:
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    if sentimiento > 0:
        sentimientos.append("positivo")
    elif sentimiento < 0:
        sentimientos.append("negativo")
    else:
        sentimientos.append("neutro")

# Imprimir los mensajes y sus sentimientos correspondientes
for i, mensaje in enumerate(mensajes):
    #print("Mensaje:", mensaje)
    print("Sentimiento:", sentimientos[i])
    print()

sentimientos


Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: positivo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento


Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento:

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: positivo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: positivo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: positivo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimi

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: negativo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: negativo

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimiento: neutro

Sentimien

['neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'positivo',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'positivo',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'negativo',
 'ne

In [17]:
# Lista para almacenar los sentimientos de cada mensaje
sentimientos = {"positivo": 0, "negativo": 0, "neutro": 0}

# Iterar sobre cada mensaje y calcular el sentimiento
for texto in mensajes:
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    if sentimiento > 0:
        sentimientos["positivo"] += 1
    elif sentimiento < 0:
        sentimientos["negativo"] += 1
    else:
        sentimientos["neutro"] += 1

# Imprimir el recuento de sentimientos
for sentimiento, count in sentimientos.items():
    print(f"Sentimiento {sentimiento}: {count}")

Sentimiento positivo: 25
Sentimiento negativo: 14
Sentimiento neutro: 1735


# vamos a probar con VADER

In [18]:
valores_de_sentimiento = {
    "feliz": 0.7,
    "bien": 0.5,
    "genial": 0.8,
    "triste": -0.7,
    "cabreado": -0.8,
    "mal": -0.5,
    "espero": 0,
    "caer": -0.1,
    "gripe": -0.1,
    "malo": -0.2,
    "vale perfecto": 0.8,
    "jajajajaja": 0.1,
}

In [19]:
# Crea una instancia del analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()

# DataFrame de Pandas con una columna 'Mensaje' que contiene los mensajes de WhatsApp
for texto in df['clean_text']:
    # Asegurarse de que el texto sea una cadena de texto
    if isinstance(texto, str):
        sentimientos = sia.polarity_scores(texto)
        print("Texto:", texto)
        print("Sentimientos:", sentimientos)
        print()
    else:
        # Si el valor no es una cadena de texto, imprimir un mensaje de advertencia o manejarlo de otra manera
        print("Mensaje no válido:", texto)

Texto: mensaje llamada cifrado extremo extremo nadie chat siquiera whatsapp poder leer l escucharlo
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: aln ironhack creer grupo capibara ironhack
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: aln ironhack t anadio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: holar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: l pasar foto ayer preparense ser bombardeo
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser bombardeo ahorrar practicamente laboratorio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: jajaja
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: gracia
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: fan icono grupo
Sentimientos: {'neg': 0.0, 'neu': 0.465, 'pos': 0.535, 'compound': 0.3182}

Texto: entregar laborato

Texto: sii haber ver
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: l pasar df decir ser atributo
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: conseguir sacar l
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ver video alejo mann volver trabajo hacer laboratorio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: si saco t decir tambien
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: sonar creer hacer sort
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser bajar gimnasio luego cerveza tampoco tener cabeza sitio manana l revisar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: jajajaj
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: pues l haber probar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: hacer bien menudo laboratorio jajajajajar

Texto: pasar igual ser dia hablar ayer posar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: chica alguno haber conseguir avanzar labs ser ser ser incapaz conectar base dato publications
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: sentir l nadar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser recolectar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: reconectar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: conectada slack
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser vacacin haber hacer dano
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: 
Sentimientos: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

Texto: s
Sentimientos: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

Texto: t escribir
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: d

Texto: ser l habiar hecho l decir victor
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser l entendi
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: l terminaste
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ver pull request
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: 
Sentimientos: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

Texto: l corrigiir rossellar good job meme correspondiente
Sentimientos: {'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}

Texto: pues entonces dar igual l hagar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: tableau public podeis ver quedar creer alguno bien hecho
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: jajaja rosello
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: si l dar igual l hagar mientras l hagar trat menos jajaja
Sentimien

Texto: jajajajajar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: terir necesitar vaca urgente
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: habeis visto mensaje rocio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: carrer week ir ser online
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: sera abril
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: siiii
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: poder ir tenerife entonces
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: capis t funcionar scipy
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: llevar minuto dar vuelta tio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: l haber intentar llegar casa
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: seguir dejar 

Texto: joe ser l hacer resumeio
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: tener pagar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: rabiar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: tener pagar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: exportar cobrar semana
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser l mejor l pagar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: l hacer ser pasar l link
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: vdd ay seguro haber opcion
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: wait quizar l pueda copiar exportar canva
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: luego t l enseno
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: og dejandono deber saba

Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: aunq l octavio dirar noooo ser ml
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ser clave capi
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: aunque ser ml tope palabro
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: holii
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: tener idea ejemplo ml
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: creer l tener mente valer
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: probar exportar dato
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: haber alguien
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: pasar pequeno capibara
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: final sacasteis csv
Sentimientos: {'neg': 0.0, 'neu'

Texto: rodri ir
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: cabron
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: habiar conseguido llegar carlos
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: rodri ser crack vdd
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: ahora bajar poquito
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: capis vai
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: creer remonte
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: jajajjajar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: holar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: quizas subir mas vdd ser dar igual quedar
Sentimientos: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Texto: subo hacer readme listo
Sentimientos: {'neg': 0.0, 'neu': 1.0,

# N-grams

In [20]:
import nltk, re, string, collections
from nltk.util import ngrams # function for making ngrams

import nltk
from nltk import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/sil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
df['clean_text']

KeyError: 'clean_text'

In [22]:
# Función para generar n-grams de una lista de tokens
def generar_ngrams(tokens, n):
    return list(ngrams(tokens, n))

# Tokenización y generación de n-grams para cada fila del DataFrame
n = 2  # Tamaño de los n-grams

def analisis_sentimiento(texto):
    # Tokenizar el texto
    tokens = word_tokenize(texto)
    # Generar n-grams
    n_grams = generar_ngrams(tokens, n)
    # Calcular el sentimiento para cada n-gram y promediar los resultados
    sentimientos = [TextBlob(' '.join(ngram)).sentiment.polarity for ngram in n_grams]
    # Calcular el promedio de los sentimientos de los n-grams
    promedio_sentimientos = sum(sentimientos) / len(sentimientos)
    return promedio_sentimientos

# Aplicar el análisis de sentimientos a cada texto en el DataFrame
df['sentimiento'] = df['clean_text'].apply(analisis_sentimiento)

print(df)

KeyError: 'clean_text'